<a href="https://colab.research.google.com/github/aftera1l/SolarLLMZeroToAll/blob/main/solar_easy_pdf_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

! pip3 install -qU langchain-upstage pypdf

In [14]:
from IPython.display import display, HTML
pip install rank_bm25
pip install tavily-python
from langchain_upstage import UpstageLayoutAnalysisLoader
import os
from openai import OpenAI
from pprint import pprint
pip install langchain-community
import warnings
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage
from langchain_community.retrievers import BM25Retriever
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings
from langchain.docstore.document import Document
from langchain_core.prompts import PromptTemplate
from tavily import TavilyClient

warnings.filterwarnings("ignore")

os.environ["UPSTAGE_API_KEY"] = 'up_5kT8zTY70eUzQoz5GCQBzBWXxuQsJ'
os.environ["TAVILY_API_KEY"] = "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"
os.environ['NEWS_API_KEY'] = "17944328643d4922b6bc5ade37224e86"

![SolarSample](https://github.com/aftera1l/SolarLLMZeroToAll/blob/main/figures/solar_sample.png?raw=1)

In [11]:
loader = PyPDFLoader("target_long.pdf")
docs = loader.load()  #보험 약관 페이지가 매우 기므로, 나중에 이 데이터를 split해서 retriever에 넣을 것이다.

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
신한 해외여행 보험  
2024년 4월부터  적용  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
이 보험약관은  금융소비자보호법  및 소비자보호  내부통제기준에  따른 절차를  거쳐 제공됩니다 . 



In [12]:

llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context.
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser() #Chain은 prompy template의 요구를 따라, Upstage의 llm 모델을 이용하여, StrOutputParser로 답변의 문장구조를 사람답게 가공한다.

In [20]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)

retriever = BM25Retriever.from_documents(splits)

In [23]:
pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 

In [24]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
)

retriever = vectorstore.as_retriever()

In [25]:
retriever.invoke("보험금 지급사유는?")

[Document(page_content='제5조(보험금을  지급하지  않는 사유) \n① 회사는  다음 중 어느 한 가지로  보험금  지급사유가  발생한 때에는  보험금을  지급하지  않습니다 . \n1. 피보험자가  고의로  자신을  해친 경우. 다만, 피보험자가  심신상실  등으로  자유로운  의사결정을  할 수 없는 상태에서  \n자신을  해친 경우에는  보험금의  지급사유에서  정한 해당 보험금을  지급합니다 . \n2. 보험수익자가  고의로  피보험자를  해친 경우. 다만, 그 보험수익자가  보험금의  일부 보험수익자인  경우에는  다른 보\n험수익자에  대한 보험금은  지급합니다 . \n3. 계약자가  고의로  피보험자를  해친 경우 \n4. 피보험자의  임신, 출산(제왕절개를  포함합니다 ), 산후기 . 그러나 , 회사가  보장하는  보험금  지급사유와  보장개시일부\n터 2년이 지난 후에 발생한  습관성  유산, 불임 및 인공수정  관련 합병증으로  인한 경우에는  보험금을  지급합니다  \n<용어풀이 > \n[습관성  유산, 불임 및 인공수정 ] \n한국표준질병 ·사인분류상의  N96~N98 에 해당하는  질병을  말합니다 . \n5. 전쟁, 외국의  무력행사 , 혁명, 내란, 사변, 폭동 \n② 회사는  다른 약정이  없으면  피보험자가  직업, 직무 또는 동호회  활동목적으로  아래에  열거된  행위로  인하여  제3조(보험\n금의 지급사유 )의 상해 관련 보험금  지급사유가  발생한  때에는  해당 보험금을  지급하지  않습니다 . \n1. 전문등반 (전문적인  등산용구를  사용하여  암벽 또는 빙벽을  오르내리거나  특수한  기술, 경험, 사전훈련을  필요로  하\n는 등반을  말합니다 ), 글라이더  조종, 스카이다이빙 , 스쿠버다이빙 , 행글라이딩 , 수상보트 , 패러글라이딩  \n2. 모터보트 , 자동차  또는 오토바이에  의한 경기, 시범, 흥행(이를 위한 연습을  포함합니다 ) 또는 시운전 (다만, 공용도로\n상에서  시운전을

In [34]:
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])



def solar_paper_search(query: str) -> str:
    """Query for research paper about solarllm, dus, llm and general AI.
    If the query is about DUS, Upstage, AI related topics, use this.
    """
    return solar_summary



def internet_search(query: str) -> str:
    """This is for query for internet search engine like Google.
    Query for general topics.
    """
    return tavily.search(query=query)



def get_news(topic: str) -> str:
    """Get latest news about a topic.
    If users are more like recent news, use this.
    """
    # https://newsapi.org/v2/everything?q=tesla&from=2024-04-01&sortBy=publishedAt&apiKey=API_KEY
    # change this to request news from a real API
    news_url = f"https://newsapi.org/v2/everything?q={topic}&apiKey={os.environ['NEWS_API_KEY']}"
    respnse = requests.get(news_url)
    return respnse.json()


tools = [solar_paper_search, internet_search, get_news]


llm_with_tools = llm.bind_tools(tools)

In [55]:
query = "보험금 지급사유는?"
a = llm_with_tools.invoke(query).tool_calls
keyword = a[0]['args'].values()
context_docs = retriever.invoke(keyword)
chain.invoke({"question": query, "Context": context_docs})

'보험금 지급사유는 상해의 직접결과로써 사망한 경우, 상해로 장해분류표에서 정한 각 장해지급률에 해당하는 장해상태가 되었을 때입니다.'

# Excercise

How can we easily get information from complicated tables for LLMs?